In [1]:
from selenium import webdriver
import pandas as pd
from bs4 import BeautifulSoup
import urllib.request as req
import pandas as pd
import requests
from io import StringIO
import requests
import time
from selenium.webdriver import Chrome
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import os
from naver_crawler import *
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import time
import re
import parser
import datetime
from urllib.request import urlopen
import webbrowser
import xmltodict
import json
from ast import literal_eval
import lxml

In [2]:
# 한국기업평가 설립일 : 1983.12.29
# 1980 ~ 2021 date_list 만들기
import time
from datetime import date,timedelta
today= date.today()
date_lst=[]
for i in range(917):
    date_lst.append((today - timedelta(days=15*i)).strftime("%Y-%m-%d"))
len(date_lst),date_lst[-1],date_lst[0]

(917, '1983-12-09', '2021-07-22')

In [25]:
table_save = table_DF.copy()

In [26]:
table_save.drop_duplicates().shape

(1952, 29)

In [27]:
table_save.Type.value_counts()

ABSTB     860
회사채       735
ABCP      173
ABS       110
기업어음       68
전자단기사채     21
Name: Type, dtype: int64

In [40]:
date_lst[3]
date_lst[4]

'2021-05-23'

In [19]:
table_DF = pd.DataFrame()
type_lst = ['회사채','기업어음','전자단기사채','ABS','ABCP','ABSTB','ICR','IFSR']
os.getcwd()

'C:\\Users\\shic\\Documents\\GitHub\\Crawling\\Desktop\\Project'

In [41]:
# 창 띄우기
delay=3
browser = Chrome()
browser.implicitly_wait(delay)

In [42]:
url = 'http://www.rating.co.kr/disclosure/QDisclosure014.do'
browser.get(url)

In [ ]:
# 1달이라는 기간동안
for t in range(len(date_lst)-3):
    start_date = browser.find_element_by_xpath('//*[@id="fromDt"]')
    start_date.clear()
    start_date.send_keys(date_lst[t+4])
    end_date = browser.find_element_by_xpath('//*[@id="toDt"]')
    end_date.clear()
    end_date.send_keys(date_lst[t+3])
    
    # 8종류의 데이터를
    for i in range(2,10):
        try:
            type_btn = browser.find_element_by_xpath('//*[@id="selectTitle"]/option[{}]'.format(i))
            type_btn.click()

            src_btn = browser.find_element_by_xpath('//*[@id="content"]/div[1]/div/div/ul/li[2]/a/span')
            src_btn.click()
        
        except:
            break
        
        # 1,11,21,... 페이지
        for k in range(4):
            
            

            # 1페이지 추출
            result = browser.page_source
            soup = BeautifulSoup(result, 'html.parser')
            table = soup.select('div.table_type4')
            table_html = str(table)

            table_df = pd.read_html(table_html)[0]
            table_df['Type']=type_lst[i-2] # Type : 1~8 가지의 종류의 데이터 이름
            table_DF=pd.concat([table_DF,table_df],axis=0)
            print("table_DF:",table_DF.shape)

            # 2~10페이지 추출
            for j in range(4,13):
                
                try:
                    next_btn = browser.find_element_by_css_selector('#paging > a:nth-child({})'.format(j))
                    next_btn.click()
                    
                    result = browser.page_source
                    soup = BeautifulSoup(result, 'html.parser')
                    table = soup.select('div.table_type4')
                    table_html = str(table)

                    # 1. 회사채
                    table_df = pd.read_html(table_html)[0]
                    table_df['Type']=type_lst[i-2]
                    print("table_df",table_df.head())
                    table_DF=pd.concat([table_DF,table_df],axis=0)
                    print("table_DF:",table_DF.shape)
                except:
                    break
                    
                
                    
            # 다음 페이지(11,21...)로 넘어가기
            print(10*k,"페이지 완료 | table_DF.shape: ",table_DF.shape)
            print(table_DF)
            
            try:
                nextpg_btn = browser.find_element_by_css_selector('#paging > a.next')
                nextpg_btn.click()
            except:
                break

table_DF: (6691, 29)
table_df           회사명  종류  구분   회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  (주)BNK금융지주  FB  정기  028  1000      AAA(안정적)     AAA (안정적)  2021.05.25   
1  (주)BNK금융지주  FB  정기  027   900      AAA(안정적)     AAA (안정적)  2021.05.25   
2  (주)BNK금융지주  FB  정기  026  1000      AAA(안정적)     AAA (안정적)  2021.05.25   
3  (주)BNK금융지주  FB  정기  023  1000      AAA(안정적)     AAA (안정적)  2021.05.25   
4  (주)BNK금융지주  FB  정기  022  1000      AAA(안정적)     AAA (안정적)  2021.05.25   

                      공시일  평가의견  보고서 Type  
0  2021.05.25 ( 13 : 32 )   NaN  NaN  회사채  
1  2021.05.25 ( 13 : 32 )   NaN  NaN  회사채  
2  2021.05.25 ( 13 : 32 )   NaN  NaN  회사채  
3  2021.05.25 ( 13 : 32 )   NaN  NaN  회사채  
4  2021.05.25 ( 13 : 32 )   NaN  NaN  회사채  
table_DF: (6711, 29)
table_df          회사명  종류 구분     회차  발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  BNK캐피탈(주)  FB  본  252-3  200           NaN     AA- (안정적)  2021.06.03   
1  BNK캐피탈(주)  FB  본  252-2  100           NaN     AA- (안정적)  2021.06.

table_DF: (6891, 29)
table_df             회사명  종류  구분                     회차   발행액 직전등급(Outlook)  \
0  오일허브코리아여수(주)  SB  정기                    005   500       A+(안정적)   
1       (주)우리은행  FB   본  우리은행25-06-이(변)01-갑-02  5000           NaN   
2       (주)우리카드  FB   본                    200   200           NaN   
3       (주)우리카드  FB   본                  199-4   500           NaN   
4       (주)우리카드  FB   본                  199-3   600           NaN   

  현재등급(Outlook)         평가일                     공시일  평가의견  보고서 Type  
0      A+ (안정적)  2021.05.31  2021.05.31 ( 15 : 54 )   NaN  NaN  회사채  
1     AAA (안정적)  2021.05.31  2021.05.31 ( 15 : 43 )   NaN  NaN  회사채  
2      AA (안정적)  2021.05.31  2021.05.31 ( 17 : 38 )   NaN  NaN  회사채  
3      AA (안정적)  2021.05.31  2021.05.31 ( 17 : 38 )   NaN  NaN  회사채  
4      AA (안정적)  2021.05.31  2021.05.31 ( 17 : 38 )   NaN  NaN  회사채  
table_DF: (6911, 29)
table_df        회사명   종류  구분                    회차   발행액 직전등급(Outlook) 현재등급(Outlook)  \
0   인천항만공사  PFB  정기 

table_DF: (7091, 29)
table_df       회사명  종류  구분                   회차   발행액 직전등급(Outlook) 현재등급(Outlook)  \
0  중소기업은행  FB  정기  기업은행(신)1906이2.5A-07  2500      AAA(안정적)     AAA (안정적)   
1  중소기업은행  FB  정기    기업은행(신)1905이3A-28  2600      AAA(안정적)     AAA (안정적)   
2  중소기업은행  FB  정기    기업은행(신)1905이3A-17  3400      AAA(안정적)     AAA (안정적)   
3  중소기업은행  FB  정기    기업은행(신)1905이3A-09  4500      AAA(안정적)     AAA (안정적)   
4  중소기업은행  FB  정기    기업은행(신)1902이3A-13  2000      AAA(안정적)     AAA (안정적)   

          평가일                     공시일  평가의견  보고서 Type  
0  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
1  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
2  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
3  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
4  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
table_DF: (7111, 29)
table_df       회사명       종류  구분                 회차   발행액 직전등급(Outlook) 현재등급(Outlook)  \
0  중소기업은행  FB(FRN)  정기  기업은행(변)2101이1A-08  2000      AAA(안정적)     AA

table_DF: (7277, 29)
table_df       회사명  종류  구분                   회차   발행액 직전등급(Outlook) 현재등급(Outlook)  \
0  중소기업은행  FB  정기  기업은행(신)1906이2.5A-07  2500      AAA(안정적)     AAA (안정적)   
1  중소기업은행  FB  정기    기업은행(신)1905이3A-28  2600      AAA(안정적)     AAA (안정적)   
2  중소기업은행  FB  정기    기업은행(신)1905이3A-17  3400      AAA(안정적)     AAA (안정적)   
3  중소기업은행  FB  정기    기업은행(신)1905이3A-09  4500      AAA(안정적)     AAA (안정적)   
4  중소기업은행  FB  정기    기업은행(신)1902이3A-13  2000      AAA(안정적)     AAA (안정적)   

          평가일                     공시일  평가의견  보고서 Type  
0  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
1  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
2  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
3  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
4  2021.05.26  2021.05.26 ( 14 : 21 )   NaN  NaN  회사채  
table_DF: (7297, 29)
table_df       회사명       종류  구분                 회차   발행액 직전등급(Outlook) 현재등급(Outlook)  \
0  중소기업은행  FB(FRN)  정기  기업은행(변)2101이1A-08  2000      AAA(안정적)     AA

table_DF: (7470, 29)
0 페이지 완료 | table_DF.shape:  (7470, 29)
    Type (Type, )                     공시일 (공시일, 공시일) 구분 (구분, 구분)     발행액 발행한도  \
0    회사채      NaN  2021.07.14 ( 16 : 53 )        NaN  본      NaN   300.0  NaN   
1    회사채      NaN  2021.07.09 ( 14 : 33 )        NaN  본      NaN   400.0  NaN   
2    회사채      NaN  2021.07.09 ( 14 : 33 )        NaN  본      NaN  1100.0  NaN   
3    회사채      NaN  2021.07.08 ( 14 : 35 )        NaN  본      NaN   200.0  NaN   
4    회사채      NaN  2021.07.08 ( 14 : 35 )        NaN  본      NaN   200.0  NaN   
..   ...      ...                     ...        ... ..      ...     ...  ...   
15  기업어음      NaN       2021.05.31(17:49)        NaN  본      NaN     NaN  NaN   
16  기업어음      NaN       2021.05.24(13:42)        NaN  본      NaN     NaN  NaN   
17  기업어음      NaN       2021.05.28(15:49)        NaN  본      NaN     NaN  NaN   
18  기업어음      NaN       2021.05.24(13:42)        NaN  본      NaN     NaN  NaN   
19  기업어음      NaN       2021.05.31(17:35)        

...

table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  302-1   900      AA-(안정적)     AA- (안정적)  2021.05.13   
3  현대건설(주)  SB  정기    300   500      AA-(안정적)     AA- (안정적)  2021.05.13   
4  현대건설(주)  SB  정기    297  1500      AA-(안정적)     AA- (안정적)  2021.05.13   

                      공시일  평가의견  보고서 Type  
0  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
1  2021.05.13 ( 17 : 33 )   NaN  NaN  회사채  
2  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
3  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
4  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
table_DF: (9237, 29)
table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  3

table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  302-1   900      AA-(안정적)     AA- (안정적)  2021.05.13   
3  현대건설(주)  SB  정기    300   500      AA-(안정적)     AA- (안정적)  2021.05.13   
4  현대건설(주)  SB  정기    297  1500      AA-(안정적)     AA- (안정적)  2021.05.13   

                      공시일  평가의견  보고서 Type  
0  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
1  2021.05.13 ( 17 : 33 )   NaN  NaN  회사채  
2  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
3  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
4  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
table_DF: (9381, 29)
table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  3

table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  302-1   900      AA-(안정적)     AA- (안정적)  2021.05.13   
3  현대건설(주)  SB  정기    300   500      AA-(안정적)     AA- (안정적)  2021.05.13   
4  현대건설(주)  SB  정기    297  1500      AA-(안정적)     AA- (안정적)  2021.05.13   

                      공시일  평가의견  보고서 Type  
0  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
1  2021.05.13 ( 17 : 33 )   NaN  NaN  회사채  
2  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
3  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
4  2021.05.13 ( 17 : 28 )   NaN  NaN  회사채  
table_DF: (9525, 29)
table_df        회사명  종류  구분     회차   발행액 직전등급(Outlook) 현재등급(Outlook)         평가일  \
0  현대건설(주)  SB  정기  302-3   700      AA-(안정적)     AA- (안정적)  2021.05.13   
1  현대건설(주)  SB  정기  302-2  1400      AA-(안정적)     AA- (안정적)  2021.05.13   
2  현대건설(주)  SB  정기  3

table_DF: (9585, 29)
table_df                 회사명 구분      회차  발행액  직전등급   현재등급         평가일  \
0  티월드제육십사차유동화전문(유)  본  제1-14회   50   NaN  AAASF  2021.05.18   
1  티월드제육십사차유동화전문(유)  본  제1-13회   50   NaN  AAASF  2021.05.18   
2  티월드제육십사차유동화전문(유)  본  제1-12회  100   NaN  AAASF  2021.05.18   
3  티월드제육십사차유동화전문(유)  본  제1-11회  100   NaN  AAASF  2021.05.18   
4  티월드제육십사차유동화전문(유)  본  제1-10회  100   NaN  AAASF  2021.05.18   

                 공시일  평가의견  보고서 Type  
0  2021.05.18(16:26)   NaN  NaN  ABS  
1  2021.05.18(16:26)   NaN  NaN  ABS  
2  2021.05.18(16:26)   NaN  NaN  ABS  
3  2021.05.18(16:26)   NaN  NaN  ABS  
4  2021.05.18(16:26)   NaN  NaN  ABS  
table_DF: (9594, 29)
0 페이지 완료 | table_DF.shape:  (9594, 29)
   Type (Type, )                     공시일 (공시일, 공시일) 구분 (구분, 구분)     발행액 발행한도  \
0   회사채      NaN  2021.07.14 ( 16 : 53 )        NaN  본      NaN   300.0  NaN   
1   회사채      NaN  2021.07.09 ( 14 : 33 )        NaN  본      NaN   400.0  NaN   
2   회사채      NaN  2021.07.09 ( 14 : 33 )        NaN

table_DF: (9710, 29)
table_df               회사명  구분    회차  발행액  직전등급  현재등급         평가일                공시일  \
0  대명동한투유동화제사차(주)  수시   제2회  160  A1SF  A1SF  2021.05.21  2021.05.21(10:45)   
1  대명동한투유동화제사차(주)  수시  제20회  160  A1SF  A1SF  2021.05.21  2021.05.21(10:45)   
2  대명동한투유동화제사차(주)  수시  제19회  160  A1SF  A1SF  2021.05.21  2021.05.21(10:45)   
3  대명동한투유동화제사차(주)  수시  제18회  160  A1SF  A1SF  2021.05.21  2021.05.21(10:45)   
4  대명동한투유동화제사차(주)  수시  제17회  160  A1SF  A1SF  2021.05.21  2021.05.21(10:45)   

   평가의견  보고서   Type  
0   NaN  NaN  ABSTB  
1   NaN  NaN  ABSTB  
2   NaN  NaN  ABSTB  
3   NaN  NaN  ABSTB  
4   NaN  NaN  ABSTB  
table_DF: (9730, 29)


In [32]:
table_DF

,Type,"(Type, )",공시일,"(공시일, 공시일)",구분,"(구분, 구분)",발행액,발행한도,보고서,"(보고서, 보고서)",...,"(평가의견, 평가의견)","(평가일, 평가일)",한도설정일,현재등급(Outlook),현재등급,"(현재등급, Outlook)","(현재등급, 등급)",회사명,회차,"(회사명, 회사명)"
0,회사채,NaN,2021.07.14 ( 16 : 53 ),NaN,본,NaN,300.0,NaN,NaN,NaN,...,NaN,NaN,NaN,BBB+ (부정적),NaN,NaN,NaN,AJ네트웍스(주),047,NaN
1,회사채,NaN,2021.07.09 ( 14 : 33 ),NaN,본,NaN,400.0,NaN,NaN,NaN,...,NaN,NaN,NaN,AAA (안정적),NaN,NaN,NaN,(주)BNK금융지주,35-2,NaN
2,회사채,NaN,2021.07.09 ( 14 : 33 ),NaN,본,NaN,1100.0,NaN,NaN,NaN,...,NaN,NaN,NaN,AAA (안정적),NaN,NaN,NaN,(주)BNK금융지주,35-1,NaN
3,회사채,NaN,2021.07.08 ( 14 : 35 ),NaN,본,NaN,200.0,NaN,NaN,NaN,...,NaN,NaN,NaN,AA- (안정적),NaN,NaN,NaN,BNK캐피탈(주),B255-2,NaN
4,회사채,NaN,2021.07.08 ( 14 : 35 ),NaN,본,NaN,200.0,NaN,NaN,NaN,...,NaN,NaN,NaN,AA- (안정적),NaN,NaN,NaN,BNK캐피탈(주),255-3,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15,ABSTB,NaN,2021.06.29(17:15),NaN,본,NaN,1150.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A1SF,NaN,NaN,더블에스삼양(주),제9회,NaN
16,ABSTB,NaN,2021.06.29(17:15),NaN,본,NaN,1150.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A1SF,NaN,NaN,더블에스삼양(주),제8회,NaN
17,ABSTB,NaN,2021.06.29(17:15),NaN,본,NaN,1150.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A1SF,NaN,NaN,더블에스삼양(주),제7회,NaN
18,ABSTB,NaN,2021.06.29(17:15),NaN,본,NaN,1150.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,A1SF,NaN,NaN,더블에스삼양(주),제6회,NaN


In [24]:
table_DF.drop_duplicates().shape

(1952, 29)

In [120]:
src_btn = browser.find_element_by_xpath('//*[@id="content"]/div[1]/div/div/ul/li[2]/a/span')
src_btn.click()

In [122]:
try:
    nextpg_btn = browser.find_element_by_css_selector('#paging > a.next')
    nextpg_btn.click()
except:
    pass

In [198]:
table_DF1.평가일.str.slice(0,4).value_counts()

2020    2838
2021    2328
2019    2222
2018    1450
2017    1108
2016     670
2015     524
2014     268
2013     210
2012     138
2011      60
2010      10
Name: 평가일, dtype: int64

In [205]:
table_DF1=table_DF1.drop_duplicates()
table_DF2=table_DF2.drop_duplicates()
table_DF3=table_DF3.drop_duplicates()
table_DF4=table_DF4.drop_duplicates()
table_DF5=table_DF5.drop_duplicates()
table_DF6=table_DF6.drop_duplicates()
table_DF7=table_DF7.drop_duplicates()
table_DF8=table_DF8.drop_duplicates()

In [209]:
print("table_DF1:",table_DF1.shape)
print("table_DF2:",table_DF2.shape)
print("table_DF3:",table_DF3.shape)
print("table_DF4:",table_DF4.shape)
print("table_DF5:",table_DF5.shape)
print("table_DF6:",table_DF6.shape)
print("table_DF7:",table_DF7.shape)
print("table_DF8:",table_DF8.shape)

table_DF1: (11620, 11)
table_DF2: (299, 9)
table_DF3: (114, 10)
table_DF4: (119, 11)
table_DF5: (25, 11)
table_DF6: (5216, 11)
table_DF7: (420, 12)
table_DF8: (16, 12)


In [212]:
os.chdir('C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data')
table_DF1.to_csv('KOREA_CREDIT_1.csv',index=False)
table_DF2.to_csv('KOREA_CREDIT_2.csv',index=False)
table_DF3.to_csv('KOREA_CREDIT_3.csv',index=False)
table_DF4.to_csv('KOREA_CREDIT_4.csv',index=False)
table_DF5.to_csv('KOREA_CREDIT_5.csv',index=False)
table_DF6.to_csv('KOREA_CREDIT_6.csv',index=False)
table_DF7.to_csv('KOREA_CREDIT_7.csv',index=False)
table_DF8.to_csv('KOREA_CREDIT_8.csv',index=False)

In [213]:
os.getcwd()

'C:\\Users\\kuri9\\Desktop\\Crawling\\Desktop\\Project\\1_Data'